In [1]:
import itertools
import numpy as np
import functools

In [2]:
%display latex

In [3]:
def same(*args):
    for arg in args:
        assert args[0] == arg
    return args[0]

In [4]:
class GaussQuad:
    def __init__(self, alphas, betas, wf):
        self.n = same(len(alphas), len(betas))
        
        self.wf = wf
    
        J = np.zeros((self.n, self.n))
        for k, alpha, beta in zip(range(self.n), alphas, betas):
            J[k, k] = alpha
            if k-1 >= 0:
                J[k-1, k] = sqrt(beta)
                J[k, k-1] = sqrt(beta)

        x = var("x")
        polys = [0, 1]
        for k, alpha, beta in zip(itertools.count(2), alphas, betas):
            poly = (x - alpha)*polys[k-1] - beta*polys[k-2]
            polys.append(poly)
        self.poly = polys[-1]
        self.poly = self.poly.full_simplify()
        
        # Eigenvectors are returned in normalized form.
        # eig.eigenvectors[vector_dimension,vector]
        eig = np.linalg.eig(J)
        eig_val = eig.eigenvalues
        eig_vec = eig.eigenvectors.T

        self.x = eig_val.tolist()
        v = eig_vec.tolist()

        self.w = []
        for k in range(self.n):
            self.w.append(betas[0]*v[k][0]**2)
    
    @property
    def sx(self):
        return var("x")
    
    @property
    def sf(self):
        return function("f")
    
    @functools.cached_property
    def quad(self):
        quad = 0
        for xv, wv in zip(self.x, self.w):
            quad += wv*(self.sf)(xv)
        return quad
    
    @functools.cached_property
    def poly_norm(self):
        coef = max(self.poly.coefficients(), key=lambda a: a[1])[0]
        poly = self.poly / coef
        return poly
        
    @property
    def rest(self):
        a = var("a")
        b = var("b")
        
        with assuming(a < b):
            rest = (
                diff((self.sf)(self.sx), self.sx, 2*self.n)
                / factorial(2*self.n)
                * integral(self.wf(self.sx)*self.poly_norm**2, self.sx, a, b)
            )
        return rest

In [5]:
def jacobi_values(n, alpha, beta):
    alphas = []
    betas = []
    
    for k in range(n):
        alphas.append(
            (beta**2 - alpha**2)
            / ((2*k + alpha + beta) * (2*k + alpha + beta + 2))
        )
        
        if k == 0:
            betas.append(
                2**(alpha + beta + 1)
                * (
                    (gamma(alpha + 1)*gamma(beta + 1))
                    / (gamma(alpha + beta + 2))
                )
            )
        elif k == 1:
            betas.append(
                (
                    4
                    * (1 + alpha)
                    * (1 + beta)
                )
                / (
                    (2 + alpha + beta)**2
                    * (3 + alpha + beta)
                )
            )
        else:
            betas.append(
                (
                    4*k
                    * (k + alpha)
                    * (k + alpha + beta)
                    * (k + beta)
                )
                / (
                    (2*k + alpha + beta - 1)
                    * (2*k + alpha + beta)**2
                    * (2*k + alpha + beta + 1)
                )
            )
    
    w = function("w")
    x = var("x")
    w(x) = (1-x)**alpha * (1+x)**beta
    
    return alphas, betas, w

In [6]:
def secant_solve(f, p0, p1, eps, max_iter=100):
    q0 = f(p0).n()
    q1 = f(p1).n()
    for i in range(max_iter):
        p = p1 - (q1 * (p1 - p0)) / (q1 - q0)
        p = p.n()
        if abs(p - p1) < eps:
            return p
        p0, q0 = p1, q1
        p1, q1 = p, f(p).n()
    raise RuntimeError(f"could not find value with satisfactory precision after {max_iter} iterations")

# Solution

In [35]:
def the_f(alpha):
    x = var("x")
    fv = function("f")
    fv(x) = (3*pi/4)**(1-alpha) * cos(3*pi/4 * x + 3*pi/4)

    gq = GaussQuad(*jacobi_values(10, 0, -alpha))
    return gq.quad.substitute_function(gq.sf, fv)

alpha = secant_solve(the_f, 0.1, 0.9, 0.00001)
display(alpha)
display(the_f(alpha).n())

0.308443779789907

8.12572498003206e-10

In [36]:
def lup_decomp(A):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    A = A.copy()
    
    P = np.eye(n)
    
    for k in range(n-1):
        i = k + np.argmax(np.abs(A[k:, k]))
        if i != k:
            A[[i, k]] = A[[k, i]]
            P[[i, k]] = P[[k, i]]
        
        lin = np.s_[k+1:n]
        
        A[lin, k] /= A[k, k]
        A[lin, lin] -= np.matmul(A[lin, [k]], A[[k], lin])
    
    L = np.tril(A)
    np.fill_diagonal(L, 1)
    
    U = np.triu(A)

    return L, U, P


def lup_solve(A, b):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    L, U, P = lup_decomp(A)
    
    Pb = np.matmul(P, b)
    
    y = np.zeros(n)
    for i in range(n):
        y[i] = (Pb[i, 0] - np.sum(L[i, :i] * y[:i])) / L[i, i]
    y = y.reshape(-1, 1)
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i, 0] - np.sum(U[i, i:] * x[i:])) / U[i, i]
    x = x.reshape(-1, 1)

    return x

In [61]:
x0, x1, x2 = 0, 1, 2
a0, a1, a2 = var("a_0 a_1 a_2")
f = function("f")

x = var("x")
poly = a0 + a1*x + a2*x**2

sol = solve([
    a0*x0**0 + a1*x0**1 + a2*x0**2 == f(x0),
    a1 + a2*2*x1**1 == diff(f(x))(x=x1),
    a1 + a2*2*x2**1 == diff(f(x))(x=x2),
], [a0, a1, a2])[0]
display(sol)

display(poly)
display(poly.substitute(sol))

[a_0 == f(0),
 a_1 == -D[0](f)(2) + 2*D[0](f)(1),
 a_2 == 1/2*D[0](f)(2) - 1/2*D[0](f)(1)]

a_2*x^2 + a_1*x + a_0

1/2*x^2*(D[0](f)(2) - D[0](f)(1)) - x*(D[0](f)(2) - 2*D[0](f)(1)) + f(0)

In [62]:
def newton_solve(f, p0, eps, max_iter=100):
    fd = diff(f)
    for _ in range(max_iter):
        p = p0 - f(p0) / fd(p0)
        p = p.n()
        if abs(p - p0) < eps:
            return p
        p0 = p
    raise RuntimeError(f"could not find value with satisfactory precision after {max_iter} iterations")

---

In [ ]:
class SpecialGaussQuad:
    def __init__(self, alphas, betas, wf):
        self.n = same(len(alphas), len(betas))
        
        self.wf = wf
    
        J = np.zeros((self.n, self.n))
        for k, alpha, beta in zip(range(self.n), alphas, betas):
            J[k, k] = alpha
            if k-1 >= 0:
                J[k-1, k] = sqrt(beta)
                J[k, k-1] = sqrt(beta)

        x = var("x")
        polys = [0, 1]
        for k, alpha, beta in zip(itertools.count(2), alphas, betas):
            poly = (x - alpha)*polys[k-1] - beta*polys[k-2]
            polys.append(poly)
        self.poly = polys[-1]
        self.poly = self.poly.full_simplify()
        
        # Eigenvectors are returned in normalized form.
        # eig.eigenvectors[vector_dimension,vector]
        eig = np.linalg.eig(J)
        eig_val = eig.eigenvalues
        eig_vec = eig.eigenvectors.T

        self.x = eig_val.tolist()
        v = eig_vec.tolist()

        self.w = []
        for k in range(self.n):
            self.w.append(betas[0]*v[k][0]**2)
    
    @property
    def sx(self):
        return var("x")
    
    @property
    def sf(self):
        return function("f")
    
    @functools.cached_property
    def quad(self):
        quad = 0
        for xv, wv in zip(self.x, self.w):
            quad += wv*(self.sf)(xv)
        return quad
    
    @functools.cached_property
    def poly_norm(self):
        coef = max(self.poly.coefficients(), key=lambda a: a[1])[0]
        poly = self.poly / coef
        return poly
        
    @property
    def rest(self):
        a = var("a")
        b = var("b")
        
        with assuming(a < b):
            rest = (
                diff((self.sf)(self.sx), self.sx, 2*self.n)
                / factorial(2*self.n)
                * integral(self.wf(self.sx)*self.poly_norm**2, self.sx, a, b)
            )
        return rest

---

In [84]:
x = var("x")
f = function("f")
phi = function("phi")
phi(x) = x - f(x) / diff(f(x), x) * (1 + f(x))
display(phi)

x |--> x - (f(x) + 1)*f(x)/diff(f(x), x)

In [66]:
alpha = var("alpha")
phi(alpha).substitute(f(alpha) == 0)

alpha

In [70]:
diff(phi(alpha), alpha).substitute(f(alpha) == 0)

0

In [76]:
d2s = diff(phi(alpha), alpha, 2).substitute(f(alpha) == 0)
d2s

diff(f(alpha), alpha, alpha)/diff(f(alpha), alpha) - 2*diff(f(alpha), alpha)

In [78]:
C = 1/factorial(2) * d2s
C

1/2*diff(f(alpha), alpha, alpha)/diff(f(alpha), alpha) - diff(f(alpha), alpha)

In [85]:
a = var("a")
fv = function("f")
fv(x) = x**2-a
fv

x |--> x^2 - a

In [86]:
phi(x).substitute_function(f, fv)

-1/2*(x^2 - a + 1)*(x^2 - a)/x + x

# Another